# Setup

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -q requirements.txt

In [1]:
%cd yolov5

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-369-g907bef2f Python-3.12.4 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4029MiB)


Setup complete ✅ (4 CPUs, 7.7 GB RAM, 175.9/912.7 GB disk)


# Dataset

Disponibilizado em https://universe.roboflow.com/tcc-xrqer/products-expiration-dates

Utilizar uma pasta chamada 'datasets' no mesmo diretório em que está 'yolov5'

Colocar arquivos .yaml abaixo no diretório /yolov5/data (ajustar path)

In [ ]:
### tcc-data-v4.yaml

# path: ../datasets/Exp-Dates.v4i.yolov5pytorch
# train: ../train/images
# val: ../valid/images
# test: ../test/images

path: <path>/datasets/Exp-Dates.v4i.yolov5pytorch/
train: train/images
val: valid/images
test: test/images

nc: 4
names: ['code', 'date', 'due', 'prod']

roboflow:
  workspace: tcc-xrqer
  project: products-expiration-dates
  version: 4
  license: Public Domain
  url: https://universe.roboflow.com/tcc-xrqer/products-expiration-dates/dataset/4

In [ ]:
### tcc-data-v4-test.yaml

# path: ../datasets/Exp-Dates.v4i.yolov5pytorch
# train: ../train/images
# val: ../valid/images
# test: ../test/images

path: <path>/datasets/Exp-Dates.v4i.yolov5pytorch/
train: train/images
val: test/images
test: test/images

nc: 4
names: ['code', 'date', 'due', 'prod']

roboflow:
  workspace: tcc-xrqer
  project: products-expiration-dates
  version: 4
  license: Public Domain
  url: https://universe.roboflow.com/tcc-xrqer/products-expiration-dates/dataset/4

Colocar arquivo .yaml abaixo no diretório /yolov5/data/hyps 

In [ ]:
### hyp-tcc-data-v4.yaml

# Ultralytics YOLOv5 🚀, AGPL-3.0 license
# Hyperparameters when using Albumentations frameworks
# python train.py --hyp hyp.tcc-data-v4.yaml
# See https://github.com/ultralytics/yolov5/pull/3882 for YOLOv5 + Albumentations Usage examples

lr0: 0.01 # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1 # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937 # SGD momentum/Adam beta1
weight_decay: 0.0005 # optimizer weight decay 5e-4
warmup_epochs: 3.0 # warmup epochs (fractions ok)
warmup_momentum: 0.8 # warmup initial momentum
warmup_bias_lr: 0.1 # warmup initial bias lr
box: 0.05 # box loss gain
cls: 0.3 # cls loss gain
cls_pw: 1.0 # cls BCELoss positive_weight
obj: 0.7 # obj loss gain (scale with pixels)
obj_pw: 1.0 # obj BCELoss positive_weight
iou_t: 0.20 # IoU training threshold
anchor_t: 4.0 # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
# this parameters are all zero since we want to use albumentation framework
fl_gamma: 0.0 # focal loss gamma (efficientDet default gamma=1.5)

degrees: 0.0 # image rotation (+/- deg)
translate: 0 # image translation (+/- fraction)
shear: 0 # image shear (+/- deg)
flipud: 0.0 # image flip up-down (probability)
fliplr: 0.0 # image flip left-right (probability)

scale: 0.5 # image scale (+/- gain)
perspective: 0.0 # image perspective (+/- fraction), range 0-0.001
mosaic: 1.0 # image mosaic (probability) #0.2 
mixup: 1.0 # image mixup (probability) # 0.2
copy_paste: 1.0 # segment copy-paste (probability) #0.2
hsv_h: 0.015 # image HSV-Hue augmentation (fraction)
hsv_s: 0.7 # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4 # image HSV-Value augmentation (fraction)

# Treinamento
Documentação Referência: https://docs.ultralytics.com/modes/train/

**Função p identificar qual a época utilizada em best.pt após treinamento**

Referência: https://github.com/ultralytics/ultralytics/issues/14137

In [ ]:
import pandas as pd

def get_best_epoch(path_to_results):

    # Load the training log
    results = pd.read_csv(path_to_results)

    # Strip spaces
    results.columns = results.columns.str.strip()
    # print(results.columns)

    # Calculate fitness
    results["fitness"] = results["metrics/mAP_0.5"] * 0.1 + results["metrics/mAP_0.5:0.95"] * 0.9

    # Find the epoch with the highest fitness
    best_epoch = results['fitness'].idxmax() + 1

    print(f"Best model was saved at epoch: {best_epoch}")

## Treinamento de 150 épocas

In [ ]:
!python train.py --epochs 150 --data tcc-data-v3.yaml --weights yolov5s.pt 

#Results saved to runs/train/exp6

In [9]:
get_best_epoch('./runs/train/exp6/results.csv')

Best model was saved at epoch: 126


## Treinamento de 300 épocas

In [ ]:
!python train.py --epochs 300 --data tcc-data-v4.yaml --weights yolov5s.pt 
# Results saved to runs/train/exp13

In [8]:
get_best_epoch('./runs/train/exp13/results.csv')

Best model was saved at epoch: 204


## Treinamento de 300 épocas com Data Augmentation

In [ ]:
!python train.py --epochs 300 --data tcc-data-v4.yaml --weights yolov5s.pt --hyp hyp.tcc-data-v4.yaml
# Results saved at /runs/train/exp15/

In [5]:
get_best_epoch('./runs/train/exp15/results.csv')

Best model was saved at epoch: 299


### Definições dos parâmetros de Data Augmentation no arquivo 'hyp.tcc-data-v4.yaml'

Documentação Referencia: https://docs.ultralytics.com/modes/train/#train-settings

In [ ]:
# Parâmetros com valores padrão
lr0: 0.01 # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1 # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937 # SGD momentum/Adam beta1
weight_decay: 0.0005 # optimizer weight decay 5e-4
warmup_epochs: 3.0 # warmup epochs (fractions ok)
warmup_momentum: 0.8 # warmup initial momentum
warmup_bias_lr: 0.1 # warmup initial bias lr
box: 0.05 # box loss gain
cls: 0.3 # cls loss gain
cls_pw: 1.0 # cls BCELoss positive_weight
obj: 0.7 # obj loss gain (scale with pixels)
obj_pw: 1.0 # obj BCELoss positive_weight
iou_t: 0.20 # IoU training threshold
anchor_t: 4.0 # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
# this parameters are all zero since we want to use albumentation framework
fl_gamma: 0.0 # focal loss gamma (efficientDet default gamma=1.5)
degrees: 0.0 # image rotation (+/- deg)
translate: 0 # image translation (+/- fraction)
shear: 0 # image shear (+/- deg)
flipud: 0.0 # image flip up-down (probability)
fliplr: 0.0 # image flip left-right (probability)

#Parâmetros modificados para o treinamento
scale: 0.5 # image scale (+/- gain)
perspective: 0.0 # image perspective (+/- fraction), range 0-0.001
mosaic: 1.0 # image mosaic (probability)
mixup: 1.0 # image mixup (probability)
copy_paste: 1.0 # segment copy-paste (probability)
hsv_h: 0.015 # image HSV-Hue augmentation (fraction)
hsv_s: 0.7 # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4 # image HSV-Value augmentation (fraction)

# Validação
Documentação Referência: https://docs.ultralytics.com/modes/val/

No script 'val.py', o valor padrão para threashold de IoU (Intersection Over Union) é 0.6

In [1]:
def best_f1(precision, recall):
    return 2 * (precision * recall) / (precision + recall)

### Validando resultados do treinamento de 150 épocas

In [3]:
!python val.py --weights runs/train/exp6/weights/best.pt  --data tcc-data-v3.yaml --batch 4 --save-txt --save-conf --iou-thres 0.6

# Results saved to runs/val/exp3 
                #  Class     Images  Instances          P          R      mAP50   WARNING ⚠️ NMS time limit 0.700s exceeded
                #    all         61        331      0.696      0.642      0.678       0.36
                #   code         61        109      0.526      0.561      0.556      0.278
                #   date         61         82      0.809       0.72      0.815      0.511
                #    due         61         65      0.829      0.646      0.684      0.354
                #   prod         61         75      0.619       0.64      0.656      0.296

val: data=/home/landreotti/tcc/expiry-date-detection-YOLO/yolov5/data/tcc-data-v3.yaml, weights=['runs/train/exp6/weights/best.pt'], batch_size=4, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=True, save_hybrid=False, save_conf=True, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-369-g907bef2f Python-3.12.4 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4029MiB)

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /home/landreotti/tcc/expiry-date-detection-YOLO/datasets/Exp-Dates
                 Class     Images  Instances          P          R      mAP50   WARNING ⚠️ NMS time limit 0.700s exceeded
                 Class     Images  Instances          P          R      mAP50   
                   all         61        331      0.696      0.642      0.678       0.36
                

### Validando resultados do treinamento de 300 épocas

In [ ]:
!python val.py --weights runs/train/exp13/weights/best.pt  --data tcc-data-v4.yaml --batch 4 --save-txt --save-conf --iou-thres 0.9

# Results saved to runs/val/exp20 (--batch 4 --save-txt --save-conf --iou-thres 0.45)
#                  Class     Images  Instances          P          R      mAP50   
#                    all         61        331      0.781      0.684       0.72      0.376
#                   code         61        108      0.629      0.611      0.573      0.292
#                   date         61         83      0.919       0.82      0.905      0.556
#                    due         61         65      0.826      0.708      0.744      0.363
#                   prod         61         75      0.749      0.597       0.66      0.293

# Results saved to runs/val/exp40 (--batch 4 --save-txt --save-conf --iou-thres 0.5)
# Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
#                  Class     Images  Instances          P          R      mAP50   
#                    all         61        331      0.779      0.648      0.687      0.361
#                   code         61        108      0.623      0.602       0.57      0.291
#                   date         61         83      0.914      0.773      0.847      0.535
#                    due         61         65      0.826      0.646      0.691      0.335
#                   prod         61         75      0.754      0.572       0.64      0.282

# Results saved to runs/val/exp19 (--batch 4 --save-txt --save-conf --iou-thres 0.6)
#                  Class     Images  Instances          P          R      mAP50   
#                    all         61        331      0.754      0.691      0.722       0.38
#                   code         61        108      0.605       0.62      0.572      0.292
#                   date         61         83      0.882      0.812      0.896      0.557
#                    due         61         65      0.802      0.723      0.751      0.369
#                   prod         61         75      0.729       0.61       0.67      0.304

# Results saved to runs/val/exp41 (--batch 4 --save-txt --save-conf --iou-thres 0.7)
# Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
#                  Class     Images  Instances          P          R      mAP50   
#                    all         61        331       0.73      0.693      0.715       0.38
#                   code         61        108       0.58       0.62      0.562       0.29
#                   date         61         83      0.839      0.817      0.884      0.557
#                    due         61         65      0.803      0.723      0.759      0.373
#                   prod         61         75      0.696      0.609      0.654      0.301

# Results saved to runs/val/exp42 (--batch 4 --save-txt --save-conf --iou-thres 0.8)
# Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
#                  Class     Images  Instances          P          R      mAP50   
#                    all         61        331      0.781      0.609      0.684      0.376
#                   code         61        108      0.677      0.556      0.544      0.287
#                   date         61         83      0.935      0.747      0.865      0.551
#                    due         61         65      0.806       0.64      0.722      0.371
#                   prod         61         75      0.706      0.493      0.606      0.295
# Speed: 0.5ms pre-process, 17.3ms inference, 6.3ms NMS per image at shape (4, 3, 640, 640)

# Results saved to runs/val/exp43 (--batch 4 --save-txt --save-conf --iou-thres 0.9)
# Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
#                  Class     Images  Instances          P          R      mAP50   
#                    all         61        331      0.543      0.546       0.52      0.302
#                   code         61        108      0.507      0.528      0.417      0.229
#                   date         61         83      0.662      0.708      0.729      0.482
#                    due         61         65      0.476      0.523      0.505      0.277
#                   prod         61         75      0.529      0.427      0.432      0.219

In [ ]:
print(f'45: {best_f1(0.919,0.82)}')  # conf 0.396  
print(f'50: {best_f1(0.914,0.773)}') # conf 0.392 
print(f'60: {best_f1(0.882,0.812)}') # conf 0.398  
print(f'70: {best_f1(0.839,0.817)}') # conf 0.398   
print(f'80: {best_f1(0.935,0.747)}') # conf 0.662   
print(f'90: {best_f1(0.662,0.708)}') # conf 0.665  

45: 0.8666820011500863
50: 0.8376075874333136
60: 0.8455537190082646
70: 0.8278538647342994
80: 0.8304934601664685
90: 0.6842277372262773


### Validando resultados do treinamento de 300 épocas + DataAug

In [ ]:
!python val.py --weights runs/train/exp15/weights/best.pt  --data tcc-data-v4.yaml --batch 4 --save-txt --save-conf --iou-thres 0.9

# Results saved to runs/val/exp22 (--batch 4 --save-txt --save-conf --iou-thres 0.45)
#                  Class     Images  Instances          P          R      mAP50   
#                    all         61        331      0.781      0.742      0.758      0.406
#                   code         61        108      0.691       0.63       0.61      0.327
#                   date         61         83      0.925      0.891      0.948       0.58
#                    due         61         65      0.738      0.738      0.765       0.39
#                   prod         61         75      0.769       0.71       0.71      0.327

# Results saved to runs/val/exp36 (--batch 4 --save-txt --save-conf --iou-thres 0.5) 
# Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
#                  Class     Images  Instances          P          R      mAP50   
#                    all         61        331      0.764      0.703      0.716      0.384
#                   code         61        108      0.674       0.62      0.587      0.316
#                   date         61         83      0.896      0.834      0.888      0.543
#                    due         61         65       0.71      0.677      0.709      0.359
#                   prod         61         75      0.776       0.68      0.679      0.318

# Results saved to runs/val/exp21 (--batch 4 --save-txt --save-conf --iou-thres 0.6) 
#                  Class     Images  Instances          P          R      mAP50   
#                    all         61        331      0.762      0.737      0.755      0.409
#                   code         61        108      0.671       0.63      0.609      0.332
#                   date         61         83      0.902       0.88      0.943      0.583
#                    due         61         65      0.717      0.738      0.765      0.395
#                   prod         61         75      0.756      0.702      0.701      0.326

# Results saved to runs/val/exp37 (--batch 4 --save-txt --save-conf --iou-thres 0.7)
# Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
#                  Class     Images  Instances          P          R      mAP50   
#                    all         61        331      0.725      0.697       0.71      0.387
#                   code         61        108      0.629       0.62      0.579      0.323
#                   date         61         83      0.889      0.831      0.881      0.544
#                    due         61         65      0.669      0.677      0.709      0.362
#                   prod         61         75      0.712       0.66      0.672      0.319

# Results saved to runs/val/exp38 (--batch 4 --save-txt --save-conf --iou-thres 0.8)
# Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
#                  Class     Images  Instances          P          R      mAP50   
#                    all         61        331      0.761      0.652      0.721      0.403
#                   code         61        108      0.676      0.519      0.552      0.314
#                   date         61         83      0.872      0.817      0.922      0.581
#                    due         61         65      0.814      0.673      0.748      0.397
#                   prod         61         75      0.682        0.6      0.661       0.32

# Results saved to runs/val/exp39 (--batch 4 --save-txt --save-conf --iou-thres 0.9)
# Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
#                  Class     Images  Instances          P          R      mAP50   
#                    all         61        331      0.573      0.567      0.588      0.344
#                   code         61        108      0.499      0.444      0.421      0.253
#                   date         61         83       0.68      0.771       0.81      0.529
#                    due         61         65      0.593      0.569      0.577       0.32
#                   prod         61         75      0.523      0.482      0.544      0.272


In [ ]:
print(f'45: {best_f1(0.925,0.891)}') # conf 0.374
print(f'50: {best_f1(0.896,0.834)}') # conf 0.370
print(f'60: {best_f1(0.902,0.88)}')  # conf 0.368
print(f'70: {best_f1(0.889,0.831)}') # conf 0.373
print(f'80: {best_f1(0.872,0.817)}') # conf 0.569
print(f'90: {best_f1(0.68,0.771)}')  # conf 0.65

# Qual foi o threashhold de confianca p alcançar esse f1? (< 0.4, > 0.3)

45: 0.9076817180616741
50: 0.8638890173410404
60: 0.8908641975308642
70: 0.8590220930232557
80: 0.8436044997039668
90: 0.722646450723639


# Detecção

Documentação Referência: https://docs.ultralytics.com/modes/predict/

No script 'detect.py', os valores padrão para threashold de IoU (Intersection Over Union) e Confiânça são 0.45 e 0.25 respectivamente

## Predição do modelo de 150 épocas

In [3]:
#!python detect.py --weights runs/train/exp6/weights/best.pt --source ../datasets/Exp-Dates.v3.yolov5pytorch/test/images
#!python detect.py --weights runs/train/exp6/weights/best.pt --source ../datasets/Exp-Dates.v3.yolov5pytorch/test/images --conf 0.75
!python detect.py --weights runs/train/exp6/weights/best.pt --source ../datasets/Exp-Dates.v3.yolov5pytorch/test/images --classes 1 --save-txt --save-crop
    # 2024-10-22
    # Results saved to runs/detect/exp9
    # 56 labels saved to runs/detect/exp9/labels

detect: weights=['runs/train/exp6/weights/best.pt'], source=../datasets/Exp-Dates.v3.yolov5pytorch/test/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=False, save_crop=True, nosave=False, classes=[1], agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-369-g907bef2f Python-3.12.4 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4029MiB)

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/63 /home/landreotti/tcc/expiry-date-detection-YOLO/datasets/Exp-Dates.v3.yolov5pytorch/test/images/2024-10-03_000000002_jpg.rf.78a17c9e6e369373a29dd3efeb77411e.jpg: 640x384 1 date, 28.3ms
image 2/63 /home/landreotti/tcc

## Predição do modelo de 300 épocas

### Predição para modelo treinado com 300 épocas

In [ ]:
!python detect.py --weights runs/train/exp13/weights/best.pt --source ../datasets/Exp-Dates.v4i.yolov5pytorch/test/images --classes 1 --save-txt --save-crop --save-conf --iou-thres 0.60 --conf-thres 0.398

# Validação nos mostra qual seria o melhor conf_threas para usar (pelo gráfico F1xConfidence)
# - Para IoU 0.Xx, a Conf que maximiza F1 é 0.Xxx (para todas as classes) 
           # 0.45                           0.396  
           # 0.50                           0.392 
           # 0.60                           0.398  
           # 0.70                           0.398   
           # 0.80                           0.662   
           # 0.90                           0.665  

# Results saved to runs/detect/exp40 -> --classes 1 --save-txt --save-crop --save-conf --iou-thres 0.45 --conf-thres 0.369
# Results saved to runs/detect/exp41 -> --classes 1 --save-txt --save-crop --save-conf --iou-thres 0.50 --conf-thres 0.392

# Results saved to runs/detect/exp47 -> --classes 1 --save-txt --save-crop --save-conf --iou-thres 0.60 --conf-thres 0.398

# Results saved to runs/detect/exp43 -> --classes 1 --save-txt --save-crop --save-conf --iou-thres 0.70 --conf-thres 0.398
# Results saved to runs/detect/exp44 -> --classes 1 --save-txt --save-crop --save-conf --iou-thres 0.80 --conf-thres 0.662
# Results saved to runs/detect/exp45 -> --classes 1 --save-txt --save-crop --save-conf --iou-thres 0.90 --conf-thres 0.665

detect: weights=['runs/train/exp13/weights/best.pt'], source=../datasets/Exp-Dates.v4i.yolov5pytorch/test/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.398, iou_thres=0.6, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=True, nosave=False, classes=[1], agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-369-g907bef2f Python-3.12.4 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4029MiB)

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/63 /home/landreotti/tcc/expiry-date-detection-YOLO/datasets/Exp-Dates.v4i.yolov5pytorch/test/images/2024-10-03_000000002_jpg.rf.78a17c9e6e369373a29dd3efeb77411e.jpg: 640x384 1 date, 62.5ms
image 2/63 /home/landreotti/t

### Predição para modelo treinado com 300 épocas + DataAug

In [ ]:
!python detect.py --weights runs/train/exp15/weights/best.pt --source ../datasets/Exp-Dates.v4i.yolov5pytorch/test/images --classes 1 --save-txt --save-crop --save-conf --iou-thres 0.90 --conf-thres 0.65

# Validação nos mostra qual seria o melhor conf_threas para usar (pelo gráfico F1xConfidence)
# - Para IoU 0.Xx, a Conf que maximiza F1 é 0.Xxx (para todas as classes) 
           # 0.45 -                         0.374
           # 0.50 -                         0.370
           # 0.60 -                         0.368
           # 0.70 -                         0.373
           # 0.80 -                         0.569
           # 0.90 -                         0.65

# Results saved to runs/detect/exp34 -> --classes 1 --save-txt --save-crop --save-conf --iou-thres 0.45 --conf-thres 0.374
# Results saved to runs/detect/exp35 -> --classes 1 --save-txt --save-crop --save-conf --iou-thres 0.50 --conf-thres 0.370

# Results saved to runs/detect/exp36 -> --classes 1 --save-txt --save-crop --save-conf --iou-thres 0.60 --conf-thres 0.368

# Results saved to runs/detect/exp37 -> --classes 1 --save-txt --save-crop --save-conf --iou-thres 0.70 --conf-thres 0.373
# Results saved to runs/detect/exp38 -> --classes 1 --save-txt --save-crop --save-conf --iou-thres 0.80 --conf-thres 0.569
# Results saved to runs/detect/exp39 -> --classes 1 --save-txt --save-crop --save-conf --iou-thres 0.90 --conf-thres 0.65

### Usando script 'val.py' no dataset de imagens teste para ver as métricas e gráficos gerados pela yolo 

300 épocas + DataAug

In [ ]:
!python val.py --weights runs/train/exp15/weights/best.pt  --data tcc-data-v4-test.yaml --iou-thres 0.6 --conf-thres 0.368 --save-txt

# Results saved to runs/val/exp49
# Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
#                  Class     Images  Instances          P          R      mAP50   
#                    all         63        374      0.783       0.71      0.761      0.454
#                   code         63        127      0.656      0.661      0.642      0.413
#                   date         63         90      0.804      0.822      0.869       0.56
#                    due         63         69      0.865      0.652      0.786      0.448
#                   prod         63         88      0.805      0.705      0.746      0.393

In [3]:
print(f'code: {best_f1(0.656      ,0.661)}')   
print(f'date: {best_f1(0.804      ,0.822)}') 
print(f'due: {best_f1(0.865      ,0.652)}')   
print(f'prod: {best_f1(0.805      ,0.705)}')  

code: 0.6584905087319666
date: 0.8129003690036901
due: 0.743546473302571
prod: 0.7516887417218544


In [ ]:
!python val.py --weights runs/train/exp15/weights/best.pt  --data tcc-data-v4-test.yaml --iou-thres 0.6 --save-txt --save-conf

# Results saved to runs/val/exp16 ('--iou-thres 0.45') 
#    Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
#                    Class     Images  Instances          P          R      mAP50   
#                    all          63        374      0.797        0.7      0.736      0.392
#                    code         63        127      0.695      0.638      0.639       0.36
#                    date         63         90      0.831      0.818      0.846      0.505
#                    due          63         69      0.863      0.652      0.752      0.376
#                    prod         63         88        0.8      0.693      0.709      0.326

# Results saved to runs/val/exp35 (--iou-thres 0.5)
#   Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
#                  Class     Images  Instances          P          R      mAP50   
#                    all         63        374      0.795      0.702      0.737      0.393
#                   code         63        127      0.693      0.646      0.643      0.362
#                   date         63         90      0.821      0.816       0.85      0.506
#                    due         63         69      0.864      0.652      0.751      0.376
#                   prod         63         88      0.801      0.693      0.705      0.326

# Results saved to runs/val/exp47 --iou-thres 0.6 --save-txt --save-conf
# Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
#                  Class     Images  Instances          P          R      mAP50   
#                    all         63        374      0.817      0.679      0.738      0.395
#                   code         63        127      0.705      0.622      0.636      0.365
#                   date         63         90      0.845        0.8      0.865       0.51
#                    due         63         69      0.918      0.651      0.746      0.379
#                   prod         63         88      0.802      0.643      0.707      0.327

300 épocas

In [ ]:
!python val.py --weights runs/train/exp13/weights/best.pt  --data tcc-data-v4-test.yaml --iou-thres 0.6 --conf-thres 0.398 --save-txt

# Results saved to runs/val/exp50
# Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
#                  Class     Images  Instances          P          R      mAP50   
#                    all         63        374      0.806      0.535      0.667      0.378
#                   code         63        127      0.655      0.504      0.573      0.342
#                   date         63         90      0.833      0.622      0.743       0.45
#                    due         63         69      0.952      0.571      0.785      0.445
#                   prod         63         88      0.783      0.443      0.569      0.278

In [2]:
print(f'code: {best_f1(0.655      ,0.504)}')   
print(f'date: {best_f1(0.833      ,0.622)}') 
print(f'due: {best_f1(0.952      ,0.571)}')   
print(f'prod: {best_f1(0.783      ,0.443)}')  

code: 0.5696635030198447
date: 0.7122006872852233
due: 0.7138437294812869
prod: 0.5658548123980425


In [ ]:
!python val.py --weights runs/train/exp13/weights/best.pt  --data tcc-data-v4-test.yaml --iou-thres 0.6 --save-txt --save-conf

# Results saved to runs/val/exp44 (--iou-thres 0.45 --save-txt --save-conf)
# Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
#                  Class     Images  Instances          P          R      mAP50   
#                    all         63        374      0.696      0.612      0.636      0.316
#                   code         63        127      0.564      0.567      0.554      0.291
#                   date         63         90      0.743      0.708       0.77      0.407
#                    due         63         69      0.861      0.652      0.689      0.346
#                   prod         63         88      0.615      0.523       0.53      0.221

# Results saved to runs/val/exp45 (--iou-thres 0.5 --save-txt --save-conf)
# Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
#                  Class     Images  Instances          P          R      mAP50   
#                    all         63        374      0.691      0.612      0.637      0.318
#                   code         63        127       0.56      0.567      0.556      0.295
#                   date         63         90      0.726      0.705      0.767      0.408
#                    due         63         69      0.861      0.652      0.696      0.349
#                   prod         63         88      0.615      0.523      0.527       0.22

# Results saved to runs/val/exp46 (--iou-thres 0.6 --save-txt --save-conf)
# Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
#                  Class     Images  Instances          P          R      mAP50   
#                    all         63        374      0.681      0.617      0.634      0.317
#                   code         63        127      0.542      0.575      0.555      0.295
#                   date         63         90      0.722       0.72      0.763      0.405
#                    due         63         69       0.86      0.652      0.698      0.349
#                   prod         63         88      0.599      0.523      0.519       0.22

